In [10]:
import os
from functools import reduce
import pandas as pd
from datetime import datetime
from dateutil.parser import parse
#from math import log, floor
#import csv
import requests
import json
#from collections import deque
import numpy as np
#from IPython.display import HTML
%matplotlib inline
import matplotlib.pyplot as plt
from config import apikey

In [26]:
# Call congress info
version =  'v1'
response = requests.get(
    'https://api.propublica.org/congress/v1/117/senate/members.json',
    headers={'X-API-Key': apikey},
).content

data = json.loads(response)

members = data['results'][0]['members']

# For Debug only,  comment out
#for member in members:
#    print(member['last_name'], member['state'], member['party'])

membersdf = pd.DataFrame(members)
stategrpcts = membersdf.groupby(['state','party'])['party'].count()


In [30]:
# new_df = membersdf[membersdf['state'] == 'CA']
# new_df.head()
# membersdf.head()
stategrpcts.head()

state  party
AK     R        2
AL     R        2
AR     R        2
AZ     D        2
CA     D        3
Name: party, dtype: int64

In [18]:
states = membersdf['state'].unique()
print(len(states))


50


In [52]:
## Assign Value for each state based on party 
party_df = stategrpcts.unstack()

#Create a series with the map 
party_df['Color'] = party_df.D.map(lambda x: 'Blue' if x >= 2 else "")
blue_party_series = party_df[party_df['Color'] == 'Blue']['Color']

party_df['Color'] = party_df.R.map(lambda x: 'Red' if x >= 2 else "")
red_party_series = party_df[party_df['Color'] == 'Red']['Color']

party_df['Color'] = party_df.ID.map(lambda x: 'White' if x >= 2 else "")
white_party_series = party_df[party_df['Color'] == 'White']['Color']

complete_party_df = pd.concat([blue_party_series, red_party_series, white_party_series])

# WHERE WE LEFT OFF --> Finish putting purple into series 
party_df['Color'] = party_df[~party_df.index.isin(complete_party_df)].map(lambda x: 'Purple')
party_df
# purple_party_series = party_df[]

# complete_party_df
# party_df['Color'] = party_df.ID.map(lambda x: 'White' if x >= 2 else "")
# party_df
# new_df = df.loc[df[“column”] == “unique variable”, :]



AttributeError: 'DataFrame' object has no attribute 'map'

In [13]:
# Call covid info
version =  'v1'
response = requests.get(
    'https://data.cdc.gov/resource/9mfq-cb36.json?created_at=2020-12-09T14:45:40.234'
).content

covid_data = json.loads(response)


coviddf = pd.DataFrame(covid_data)
coviddf.head()

,submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death,created_at,consent_cases,consent_deaths
0,2020-12-08T00:00:00.000,OK,205999,171497.0,34502,0.0,0,1752,1680.0,72,0.0,0,2020-12-09T14:45:40.234,Agree,Agree
1,2020-12-08T00:00:00.000,NYC,348211,315788.0,32423,4567.0,1671,24411,19694.0,4717,-5.0,-8,2020-12-09T14:45:40.234,Agree,Agree
2,2020-12-08T00:00:00.000,AR,174325,NaN,NaN,2283.0,846,2752,NaN,NaN,39.0,3,2020-12-09T14:45:40.234,Not agree,Not agree
3,2020-12-08T00:00:00.000,NY,385232,NaN,NaN,5874.0,0,10440,NaN,NaN,58.0,0,2020-12-09T14:45:40.234,Not agree,Not agree
4,2020-12-08T00:00:00.000,OH,510018,470721.0,39297,25721.0,13993,7103,6601.0,502,81.0,24,2020-12-09T14:45:40.234,Agree,Agree


In [14]:
coviddf.count()

submission_date    57
state              57
tot_cases          57
conf_cases         32
prob_cases         32
new_case           57
pnew_case          57
tot_death          57
conf_death         33
prob_death         33
new_death          57
pnew_death         57
created_at         57
consent_cases      52
consent_deaths     52
dtype: int64

In [15]:
covidstatesdf = coviddf[coviddf['state'].isin(states)][['state','tot_cases','tot_death']]
covidstatesdf.count()

state        47
tot_cases    47
tot_death    47
dtype: int64

In [ ]:
covidstatesdf.head()